## Cholesky LU (not needed right now)

In [283]:
import numpy as np

np.set_printoptions(precision=5, suppress=True)

def is_positive_definite(A):
    return np.all(np.linalg.eigvals(A) > 0)

def cholesky_LU(A):
    """
    Compute Cholesky decomposition of a positive definite matrix A = L @ L.T
    Returns lower triangular matrix L
    """
    if not isinstance(A, np.ndarray):
        A = np.array(A)
    
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix must be square")
        
    if not np.allclose(A, A.T) or not is_positive_definite(A):
        raise ValueError("Matrix must be symmetric positive definite")

    n = A.shape[0]
    L = np.zeros_like(A, dtype=float)

    for i in range(n):
        for j in range(i + 1):
            if i == j:
                L[i,i] = np.sqrt(A[i,i] - np.sum(L[i,:i]**2))
            else:
                L[i,j] = (A[i,j] - np.sum(L[i,:j]*L[j,:j])) / L[j,j]
    
    return L

# Example usage:
if __name__ == "__main__":
    A = np.array([[4, 2, -2], 
                  [2, 10, 2], 
                  [-2, 2, 5]])
    L = cholesky_LU(A)
    L_true = np.linalg.cholesky(A, upper=False)
    print("L:\n", L)
    print("Verification L@L.T:\n", L @ L.T)
    print("L:\n", L_true)

L:
 [[ 2.       0.       0.     ]
 [ 1.       3.       0.     ]
 [-1.       1.       1.73205]]
Verification L@L.T:
 [[ 4.  2. -2.]
 [ 2. 10.  2.]
 [-2.  2.  5.]]
L:
 [[ 2.       0.       0.     ]
 [ 1.       3.       0.     ]
 [-1.       1.       1.73205]]


## **Band matrix generation**

In [284]:
def generate_base_symmetric_matrix(N: int, L: int) -> np.ndarray:
    """
    Generate base symmetric band matrix
    Args:
        N: Matrix dimension
        L: Bandwidth
        rng: Random number generator
    """
    rng = np.random.default_rng()
        
    A = np.zeros((N, L), dtype=np.float64)
    
    # Generate random values for band matrix
    for i in range(L):
        values = rng.uniform(-10.0, 10.0, N-i)
        A[0:N-i, i] = values
        
    return A

print(generate_base_symmetric_matrix(3, 3))


[[-5.62691 -9.13747  4.94983]
 [-1.6631  -5.96663  0.     ]
 [ 6.00124  0.       0.     ]]


### *Conditioning*

In [285]:
def generate_well_conditioned(N: int, L: int, scale: float = 100.0) -> np.ndarray:
    """
    Generate well-conditioned symmetric matrix by:
    1. Creating base symmetric matrix
    2. Scaling diagonal elements to ensure diagonal dominance
    """
    A = generate_base_symmetric_matrix(N, L)
    # Scale first column (diagonal) to ensure well-conditioning
    A[:, 0] = A[:, 0] + scale
    return A


def generate_ill_conditioned(N: int, L: int, k: float = 2.0) -> np.ndarray:
    """
    Generate ill-conditioned symmetric matrix by:
    1. Creating base symmetric matrix
    2. Scaling first column by small factor 10^(-k)
    """
    A = generate_base_symmetric_matrix(N, L)
    # Scale first column to make matrix ill-conditioned
    A[:, 0] = A[:, 0] * (10 ** (-k))
    return A


def band_to_full_matrix(band_matrix: np.ndarray) -> np.ndarray:
    """
    Convert band matrix stored in N×L format to full N×N symmetric matrix

    Args:
        band_matrix: Band matrix in N×L format where N is dimension and L is bandwidth

    Returns:
        Full N×N symmetric matrix
    """
    N, L = band_matrix.shape
    full_matrix = np.zeros((N, N), dtype=band_matrix.dtype)

    # Fill band elements
    for i in range(N):
        for j in range(L):
            if j == 0:
                # Diagonal element
                full_matrix[i, i] = band_matrix[i, 0]
            elif i + j < N:
                # Upper band and symmetric lower band
                full_matrix[i, i + j] = band_matrix[i, j]
                full_matrix[i + j, i] = band_matrix[i, j]  # Symmetry

    return full_matrix


# def lower_band_to_full(band_matrix: np.ndarray) -> np.ndarray:
#     """
#     Convert lower triangular band matrix stored in N×L format to full N×N matrix

#     Args:
#         band_matrix: Lower triangular band matrix in N×L format
#                     where N is dimension and L is bandwidth
#     Returns:
#         Full N×N lower triangular matrix
#     """
#     N, L = band_matrix.shape
#     full_matrix = np.zeros((N, N), dtype=band_matrix.dtype)

#     for i in range(N):
#         for j in range(L):
#             if j == 0:
#                 # Diagonal element
#                 full_matrix[i, i] = band_matrix[i, 0]
#             else:
#                 if i + j < N:  # Check bounds
#                     # Map band format to lower triangular
#                     full_matrix[i + j, i] = band_matrix[i, j]

#     return full_matrix

def lower_band_to_full(band_matrix: np.ndarray) -> np.ndarray:
    N, L = band_matrix.shape
    full_matrix = np.zeros((N, N), dtype=band_matrix.dtype)
    
    # Fill diagonal from first column
    for i in range(N):
        full_matrix[i,i] = band_matrix[i,0]
    
    # Fill bands
    for j in range(1, L):  # For each band
        band_idx = 0  # Index within current band
        for i in range(j, N):  # Fill current band
            full_matrix[i,band_idx] = band_matrix[i,j]
            band_idx += 1
            
    return full_matrix

def upper_band_to_full(band_matrix: np.ndarray) -> np.ndarray:
    """
    Convert upper triangular band matrix stored in N×L format to full N×N matrix

    Args:
        band_matrix: Upper triangular band matrix in N×L format
                    where N is dimension and L is bandwidth
    Returns:
        Full N×N upper triangular matrix
    """
    N, L = band_matrix.shape
    full_matrix = np.zeros((N, N), dtype=band_matrix.dtype)

    # Fill band elements
    for i in range(N):
        for j in range(L):
            if j == 0:
                # Diagonal element
                full_matrix[i, i] = band_matrix[i, 0]
            else:
                # Upper band elements only if within matrix bounds
                if i + j < N:
                    full_matrix[i, i + j] = band_matrix[i, j]

    return full_matrix


wellc = generate_well_conditioned(3, 2)
illc = generate_ill_conditioned(3, 2)
print(illc)
print()
print(np.linalg.cond(band_to_full_matrix(illc)))
print()
print(wellc)
print()
print(np.linalg.cond(band_to_full_matrix(wellc)))
print()


[[-0.02119 -0.66724]
 [ 0.00612 -4.08009]
 [-0.06137  0.     ]]

187.17088027402667

[[102.72933   7.01727]
 [ 93.01125  -2.22168]
 [105.62748   0.     ]]

1.2030734941537788



## BC decomposition

In [286]:

def BC_decomp(A):
    """
    Performs Doolittle LU decomposition of matrix A where:
    - B is lower triangular
    - C is upper triangular with ones on diagonal
    - A = B @ C
    """
    if not isinstance(A, np.ndarray):
        A = np.array(A, dtype=float)
    
    n = A.shape[0]
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix must be square")

    B = np.zeros((n, n))
    C = np.eye(n, n)
    
    for i in range(n):
        # Calculate B elements
        for j in range(i + 1):
            sum_b = sum(B[i,k] * C[k,j] for k in range(j))
            B[i,j] = A[i,j] - sum_b
            
        # Calculate C elements
        for j in range(i + 1, n):
            sum_c = sum(B[i,k] * C[k,j] for k in range(i))
            C[i,j] = (A[i,j] - sum_c) / B[i,i]
            # C[i,j] = (A[i,j] - sum_c)
            
    return B, C

dim = 4
A = np.random.default_rng().random(size=(dim, dim), dtype=np.float64)
for i in range(dim):
  for j in range(dim):
    if i < j:
      A[i,j] = A[j, i]

B, C = BC_decomp(A)
print("B (lower):\n", B)
print("\nC (upper):\n", C)
print("\nVerification B@C:\n", B @ C)
print("\nOriginal A:\n", A)

B (lower):
 [[ 0.72926  0.       0.       0.     ]
 [ 0.0928   0.50581  0.       0.     ]
 [ 0.17608  0.28175  0.53173  0.     ]
 [ 0.66403  0.45416  0.06667 -0.38663]]

C (upper):
 [[1.      0.12726 0.24145 0.91056]
 [0.      1.      0.55703 0.89789]
 [0.      0.      1.      0.12538]
 [0.      0.      0.      1.     ]]

Verification B@C:
 [[0.72926 0.0928  0.17608 0.66403]
 [0.0928  0.51762 0.30416 0.53866]
 [0.17608 0.30416 0.73118 0.47997]
 [0.66403 0.53866 0.47997 0.63414]]

Original A:
 [[0.72926 0.0928  0.17608 0.66403]
 [0.0928  0.51762 0.30416 0.53866]
 [0.17608 0.30416 0.73118 0.47997]
 [0.66403 0.53866 0.47997 0.63414]]


## solve BC-decomposed system

In [287]:
def solve_bc_system(B: np.ndarray, C: np.ndarray, f: np.ndarray) -> np.ndarray:
    """
    Solve system Ax=f where A=BC using:
    1. By=f -> find y
    2. Cx=y -> find x 
    """
    if not isinstance(B, np.ndarray) or not isinstance(C, np.ndarray):
        raise ValueError("Inputs must be numpy arrays")
        
    n = B.shape[0]
    if B.shape[1] != n or C.shape[0] != n or C.shape[1] != n:
        raise ValueError("Invalid matrix dimensions")

    # Step 1: Solve By = f
    y = np.zeros(n)
    for i in range(n):
        y[i] = f[i]
        for k in range(i):
            y[i] -= B[i,k] * y[k]
        y[i] /= B[i,i]

    # Step 2: Solve Cx = y 
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = y[i]
        for k in range(i+1, n):
            x[i] -= C[i,k] * x[k]
        # Note: C diagonal elements are 1
        
    return x

f_test = np.ones(dim)

print(solve_bc_system(*BC_decomp(A), f_test))

print(np.linalg.solve(A, f_test))

[-0.39547 -0.11979  0.27612  1.88379]
[-0.39547 -0.11979  0.27612  1.88379]


In [288]:
def solve_band_bc_system(B: np.ndarray, C: np.ndarray, f: np.ndarray) -> np.ndarray:
    """
    Solve Ax=f where A=BC, with B and C stored as band matrices
    
    Args:
        B: Lower triangular band matrix (N×L)
        C: Upper triangular band matrix with ones on diagonal (N×L)
        f: Right-hand side vector
        
    Returns:
        x: Solution vector
    """
    N, L = B.shape
    y = np.zeros(N)
    x = np.zeros(N)
    
    # Step 1: Solve By = f (forward substitution)
    for i in range(N):
        y[i] = f[i]
        for j in range(max(0, i-L+1), i):
            y[i] -= B[j,(i-j)] * y[j]
        y[i] /= B[i,0]  # Diagonal elements in first column
            
    # Step 2: Solve Cx = y (backward substitution)
    x[N-1] = y[N-1]  # C has ones on diagonal
    for i in range(N-2, -1, -1):
        x[i] = y[i]
        for j in range(1, min(L, N-i)):
            x[i] -= C[i,j] * x[i+j]
            
    return x

## band BC decomposition

In [289]:
def band_BC_decomp(A: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Decompose symmetric band matrix A into B*C where:
    - B is lower triangular
    - C is upper triangular with ones on diagonal
    - A is stored in N×L format where N is dimension and L is bandwidth

    Args:
        A: Band matrix stored as N×L array
    Returns:
        B: Lower triangular matrix
        C: Upper triangular matrix with ones on diagonal
    """
    N, L = A.shape
    B = np.zeros((N, N))
    C = np.eye(N)

    for j in range(N):
        # Process diagonal and below
        for i in range(j, min(j + L, N)):
            if i == j:
                # Diagonal element
                B[i, j] = A[i, 0] - sum(B[i, k] * C[k, j] for k in range(j))
            else:
                # Lower band
                B[i, j] = A[j, i - j] - sum(B[i, k] * C[k, j] for k in range(j))

        # Process above diagonal
        for i in range(j + 1, min(j + L, N)):
            # Upper band
            C[j, i] = (A[j, i - j] - sum(B[j, k] * C[k, i] for k in range(j))) / B[j, j]

    return B, C


mat_1 = generate_well_conditioned(dim, dim - 1)
mat_1_full = band_to_full_matrix(mat_1)
print(mat_1)
print()
print(mat_1_full)
print()

B, C = band_BC_decomp(mat_1)

print(B)
print()
print(C)
print()
print(B @ C)


[[ 93.40919   3.52219  -0.13089]
 [100.06782  -2.93735  -8.07455]
 [104.99003  -1.39254   0.     ]
 [104.97606   0.        0.     ]]

[[ 93.40919   3.52219  -0.13089   0.     ]
 [  3.52219 100.06782  -2.93735  -8.07455]
 [ -0.13089  -2.93735 104.99003  -1.39254]
 [  0.       -8.07455  -1.39254 104.97606]]

[[ 93.40919   0.        0.        0.     ]
 [  3.52219  99.935     0.        0.     ]
 [ -0.13089  -2.93242 104.9038    0.     ]
 [  0.       -8.07455  -1.62948 104.29834]]

[[ 1.       0.03771 -0.0014   0.     ]
 [ 0.       1.      -0.02934 -0.0808 ]
 [ 0.       0.       1.      -0.01553]
 [ 0.       0.       0.       1.     ]]

[[ 93.40919   3.52219  -0.13089   0.     ]
 [  3.52219 100.06782  -2.93735  -8.07455]
 [ -0.13089  -2.93735 104.99003  -1.39254]
 [  0.       -8.07455  -1.39254 104.97606]]


In [290]:
def band_BC_decomp_band_storage(A: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Decompose symmetric band matrix A into B*C where both B and C stored in band format
    
    Args:
        A: Band matrix stored as N×L array
    Returns:
        B: Lower triangular band matrix (N×L)
        C: Upper triangular band matrix with ones on diagonal (N×L)
    """
    N, L = A.shape
    B = np.zeros((N, L))
    C = np.zeros((N, L))
    
    # Set ones on diagonal of C
    C[:, 0] = 1.0
    
    for j in range(N):
        # Process diagonal and below
        for i in range(j, min(j + L, N)):
            if i == j:
                # Diagonal element
                sum_b = sum(B[i,k] * C[k,0] for k in range(j))
                B[i,0] = A[i,0] - sum_b
            else:
                # Lower band
                diff = i - j
                sum_b = sum(B[i,k] * C[k,diff] for k in range(j))
                B[i,diff] = A[j,diff] - sum_b
                
        # Process above diagonal
        for i in range(j + 1, min(j + L, N)):
            diff = i - j
            # Upper band
            sum_c = sum(B[j,k] * C[k,diff] for k in range(j))
            C[j,diff] = (A[j,diff] - sum_c) / B[j,0]
            
    return B, C

mat_2 = generate_well_conditioned(dim, dim)
mat_2_full = band_to_full_matrix(mat_2)
print(mat_2)
print()
print(mat_2_full)
print()

B, C = band_BC_decomp_band_storage(mat_2)

print(B)
print()
print(C)
print()
print(lower_band_to_full(B))
print()
print(upper_band_to_full(C))
print()
print(lower_band_to_full(B) @ upper_band_to_full(C))

[[ 98.18837  -1.6567    9.86638   3.85015]
 [100.73598   8.99369   0.44165   0.     ]
 [108.93584   5.79708   0.        0.     ]
 [101.47803   0.        0.        0.     ]]

[[ 98.18837  -1.6567    9.86638   3.85015]
 [ -1.6567  100.73598   8.99369   0.44165]
 [  9.86638   8.99369 108.93584   5.79708]
 [  3.85015   0.44165   5.79708 101.47803]]

[[ 98.18837   0.        0.        0.     ]
 [100.73598  -1.6567    0.        0.     ]
 [ 99.94216   8.99369   9.86638   0.     ]
 [ 95.23931   5.79708   0.44165   3.85015]]

[[ 1.      -0.01687  0.10048  0.03921]
 [ 1.       0.10615 -0.0961   0.     ]
 [ 1.       0.06532  0.       0.     ]
 [ 1.       0.       0.       0.     ]]

[[ 98.18837   0.        0.        0.     ]
 [ -1.6567  100.73598   0.        0.     ]
 [  9.86638   8.99369  99.94216   0.     ]
 [  3.85015   0.44165   5.79708  95.23931]]

[[ 1.      -0.01687  0.10048  0.03921]
 [ 0.       1.       0.10615 -0.0961 ]
 [ 0.       0.       1.       0.06532]
 [ 0.       0.       0.      

In [291]:
f_test_2 = np.ones(dim)

B1, C1 = band_BC_decomp_band_storage(A)
B2, C2 = BC_decomp(A)

print(f"A: \n", A)
print(f"B1: \n", lower_band_to_full(B1))
print(f"B2: \n", B2)
print(f"C1: \n", upper_band_to_full(C1))
print(f"C2: \n", C2)
print(f"B1@C1: \n", lower_band_to_full(B1)@upper_band_to_full(C1))
print(f"B2@C2: \n", B2@C2)

# print(solve_band_bc_system(*band_BC_decomp_band_storage(A), f_test_2))

# print(solve_bc_system(*BC_decomp(A), f_test_2))

# print(np.linalg.solve(A, f_test_2))


A: 
 [[0.72926 0.0928  0.17608 0.66403]
 [0.0928  0.51762 0.30416 0.53866]
 [0.17608 0.30416 0.73118 0.47997]
 [0.66403 0.53866 0.47997 0.63414]]
B1: 
 [[ 0.72926  0.       0.       0.     ]
 [ 0.0928   0.0928   0.       0.     ]
 [ 0.17608  0.51762 -0.34154  0.     ]
 [ 0.66403  0.30416  0.30416  0.05572]]
B2: 
 [[ 0.72926  0.       0.       0.     ]
 [ 0.0928   0.50581  0.       0.     ]
 [ 0.17608  0.28175  0.53173  0.     ]
 [ 0.66403  0.45416  0.06667 -0.38663]]
C1: 
 [[1.      0.12726 0.24145 0.91056]
 [0.      1.      5.45023 3.03594]
 [0.      0.      1.      7.24222]
 [0.      0.      0.      1.     ]]
C2: 
 [[1.      0.12726 0.24145 0.91056]
 [0.      1.      0.55703 0.89789]
 [0.      0.      1.      0.12538]
 [0.      0.      0.      1.     ]]
B1@C1: 
 [[ 0.72926  0.0928   0.17608  0.66403]
 [ 0.0928   0.10461  0.52821  0.36625]
 [ 0.17608  0.54002  2.5221  -0.74173]
 [ 0.66403  0.38866  2.1222   3.78652]]
B2@C2: 
 [[0.72926 0.0928  0.17608 0.66403]
 [0.0928  0.51762 0.3041